In [1]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests as req
import requests

from urllib.request import urlopen
import re

import time
import sys

In [53]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Getting books and reviews directly from goodreads.

In [2]:
from importlib import reload  
import src.helper_functions as hf 
hf = reload(hf)

In [202]:
# b_links = []

# page = pages[25]
# r = requests.get(page)
# soup = bs(r.content, 'html')
# divs = soup.find_all('div', {"class": "elementList"})[:50]
# links_for_books = ['http://goodreads.com/' + divs[i].find_all("a", {'class':"bookTitle"})[0].get('href') for i in range(50)]
# b_links.extend(links_for_books)


In [78]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science?page=1'
pages = [f'https://www.goodreads.com/shelf/show/popular-science?page={x}' for x in range(1, 26)]

In [79]:
pages

['https://www.goodreads.com/shelf/show/popular-science?page=1',
 'https://www.goodreads.com/shelf/show/popular-science?page=2',
 'https://www.goodreads.com/shelf/show/popular-science?page=3',
 'https://www.goodreads.com/shelf/show/popular-science?page=4',
 'https://www.goodreads.com/shelf/show/popular-science?page=5',
 'https://www.goodreads.com/shelf/show/popular-science?page=6',
 'https://www.goodreads.com/shelf/show/popular-science?page=7',
 'https://www.goodreads.com/shelf/show/popular-science?page=8',
 'https://www.goodreads.com/shelf/show/popular-science?page=9',
 'https://www.goodreads.com/shelf/show/popular-science?page=10',
 'https://www.goodreads.com/shelf/show/popular-science?page=11',
 'https://www.goodreads.com/shelf/show/popular-science?page=12',
 'https://www.goodreads.com/shelf/show/popular-science?page=13',
 'https://www.goodreads.com/shelf/show/popular-science?page=14',
 'https://www.goodreads.com/shelf/show/popular-science?page=15',
 'https://www.goodreads.com/shelf/

In [149]:
def gr_sign_in(driver):
    login_page = 'https://www.goodreads.com'
    driver.get(login_page)

    username = driver.find_element_by_id("userSignInFormEmail")
    username.clear()
    username.send_keys("dilkinn@gmail.com")
    time.sleep(1)

    password = driver.find_element_by_name("user[password]")
    password.clear()
    password.send_keys("pa4a4ok73")
    time.sleep(1)

    driver.find_element_by_xpath("//input[contains(@value,'Sign in')]").click()
    time.sleep(1)
    return driver

In [150]:
## book_links

def scrap_book_links(page1):
    driver = webdriver.Chrome('chromedriver')
    driver = gr_sign_in(driver)
    arr_all = []
    
    driver.get(page1)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_book_links_from_soup(soup)
    arr_all.extend(arr)
    
    while True:
        try:
            print("next_page")
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            time.sleep(1)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_book_links_from_soup(soup)
            arr_all.extend(arr)           
        except:
            driver.close()
            break
            
    print(len(arr_all))
    np.array(arr_all)
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/book_links_goodreads.csv')
    return df

def get_book_links_from_soup(soup):
    b_links = []
    divs = soup.find_all('a', {"class": "leftAlignedImage"})
    for div in divs:
        b_links.append('http://goodreads.com' + div.get('href'))
    return b_links

In [151]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science'
book_links_df = scrap_book_links(popular_science_link)

next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
next_page
1291


In [155]:
b_l = list(book_links_df[0])

In [156]:
b_l[0]

'http://goodreads.com/book/show/3869.A_Brief_History_of_Time'

# Get information about the books

In [158]:
def get_the_soup(book_link):
    driver = webdriver.Chrome('chromedriver')
    driver = gr_sign_in(driver)
    driver.get(book_link)
    soup = bs(driver.page_source, 'html.parser')
    driver.close()
    return soup

def get_book_info(book_link, soup):
    isbn_str = soup.find_all('div',{"class":"infoBoxRowItem"})[1].get_text()

    isbn_str = isbn_str.replace('\n', '')
    isbn_str = isbn_str.replace('(', '')
    isbn_str = isbn_str.replace(')', '')
    isbn, isbn13 = isbn_str.strip().split()[0], isbn_str.strip().split()[2]

    title_str = soup.find_all('h1',{"id":"bookTitle"})[0].get_text()
    title_str = title_str.replace('\n', "").strip()

    original_title = soup.find_all('div',{"id":"bookDataBox"})[0].find_all('div',{'class':'infoBoxRowItem'})[0].get_text()

    avg_rating = soup.find_all('span',{"itemprop":"ratingValue"})[0].get_text()
    avg_rating = avg_rating.replace('\n', '').strip()
    avg_rating = float(avg_rating)

    author_link = soup.find_all('a',{"class":"authorName"})[0].get('href')
    spans = soup.find_all('a',{"class":"authorName"})[0].find_all('span',{'itemprop':'name'})

    authors_names = []
    for span in spans:
        authors_names.append(span.get_text())
    authors_names

    book_description = soup.find_all('div',{"id":"description"})[0].find_all('span')[1].get_text()
    
    dct = {'book_link': book_link, 'isbn': isbn, 'isbn13': isbn13, 
          'book_title' : title_str, 'original_title' : original_title,
          'avg_rating' : avg_rating, 'author_link': author_link,
          'author_names': authors_names, 'book_description': book_description}
    
    return dct

In [159]:
def get_reviews_array_from_soup(soup):
    arr = []
    divs = soup.find_all('div',{"id":"bookReviews"})[0].find_all('div',{'class':'friendReviews elementListBrown'})
    
    for div in divs:
        user_link = 'http://goodreads.com' + div.find_all('span',{'itemprop':'author'})[0].find('a').get('href')
        ranking = len(div.find_all('span', {'class':'staticStar p10'}))
        try:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[1].get_text()
        except:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[0].get_text()
        dct = {"isbn":isbn, 'user_link': user_link, 'ranking':ranking, "review": review}
        arr.append(dct)
    return arr

def get_reviews_from_book_link(isbn, b_l):
    driver = webdriver.Chrome('chromedriver')
    driver = gr_sign_in(driver)
    arr_all = []
    
    driver.get(b_l)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_reviews_array_from_soup(soup)
    arr_all.extend(arr)
    
    while True:
        try:
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            time.sleep(1)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_reviews_array_from_soup(soup)
            arr_all.extend(arr)           
        except:
            driver.close()
            break
            
    print(len(arr_all))
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/revs_{isbn}.csv')
    return df

In [163]:
len(b_l)

1291

In [165]:
for i in range(0,1200,100):
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100


In [ ]:
books_arr = []
failed_books_list = []
for n in range(0, 1200, 100):
    for i, book_l in enumerate(b_l[n:n+100]):
        sys.stdout.write("\r" + f'book n : {i}\n')
        sys.stdout.flush()
        try:
            soup = get_the_soup(book_l)
            dct = get_book_info(book_l, soup)
            isbn = dct['isbn']
            if len(isbn) == 10:
                reviews = get_reviews_from_book_link(isbn,book_l)
                books_arr.append(dct)
                df = pd.DataFrame(books_arr)
                df.to_csv(f'../data/gr_books_df_{n}.csv')
            else:
                failed_books_list.append(bool_l)
        except:
            failed_books_list.append(book_l)
            continue

book n : 0
book n : 1
book n : 2
book n : 3
book n : 4
570
book n : 5
book n : 6


In [161]:
# selenium

In [51]:
driver.close()

In [66]:
l = []
l.extend(['ds','sd','f'])

In [67]:
l

['ds', 'sd', 'f']

In [75]:
len(failed_books_list)

150

In [76]:
df

,book_link,isbn,isbn13,book_title,original_title,avg_rating,author_link,author_names,book_description
0,http://goodreads.com//book/show/3869.A_Brief_H...,0553380168,9780553380163,A Brief History of Time,A Brief History of Time: From the Big Bang to ...,4.18,https://www.goodreads.com/author/show/1401.Ste...,[Stephen Hawking],In the ten years since its publication in 1988...
1,http://goodreads.com//book/show/21.A_Short_His...,076790818X,9780767908184,A Short History of Nearly Everything,A Short History of Nearly Everything,4.20,https://www.goodreads.com/author/show/7.Bill_B...,[Bill Bryson],"In Bryson's biggest book, he confronts his gre..."
2,http://goodreads.com//book/show/61535.The_Self...,0199291152,9780199291151,The Selfish Gene,The Selfish Gene,4.14,https://www.goodreads.com/author/show/1194.Ric...,[Richard Dawkins],The Selfish Gene: 30th Anniversary Edition—wit...
3,http://goodreads.com//book/show/55030.Cosmos,0375508325,9780375508325,Cosmos,Cosmos,4.37,https://www.goodreads.com/author/show/10538.Ca...,[Carl Sagan],"Cosmos has 13 heavily illustrated chapters, co..."
4,http://goodreads.com//book/show/63697.The_Man_...,0684853949,9780684853949,The Man Who Mistook His Wife for a Hat and Oth...,The Man Who Mistook His Wife for a Hat and Oth...,4.06,https://www.goodreads.com/author/show/843200.O...,[Oliver Sacks],"If a man has lost a leg or an eye, he knows he..."
...,...,...,...,...,...,...,...,...,...
1095,http://goodreads.com//book/show/11256979-the-m...,1439192812,9781439192818,The Magic of Reality: How We Know What's Reall...,The Magic of Reality: How We Know What's Reall...,4.04,https://www.goodreads.com/author/show/1194.Ric...,[Richard Dawkins],Magic takes many forms. Supernatural magic is ...
1096,http://goodreads.com//book/show/27213168-i-con...,0062368591,9780062368591,I Contain Multitudes: The Microbes Within Us a...,I Contain Multitudes,4.15,https://www.goodreads.com/author/show/4124738....,[Ed Yong],"Every animal, whether human, squid, or wasp, i..."
1097,http://goodreads.com//book/show/17910054-the-s...,0805092994,9780805092998,The Sixth Extinction: An Unnatural History,The Sixth Extinction: An Unnatural History,4.13,https://www.goodreads.com/author/show/45840.El...,[Elizabeth Kolbert],"Over the last half-billion years, there have b..."
1098,http://goodreads.com//book/show/2094.A_Briefer...,0553804367,9780553804362,A Briefer History of Time,A Briefer History of Time,4.22,https://www.goodreads.com/author/show/1401.Ste...,[Stephen Hawking],"Stephen Hawking's worldwide bestseller, A Brie..."


In [77]:
df.to_csv('../data/gr_books_df.csv')